## LSTM model ##

- This is an exploratory notebook to find the best hyperparameters for the model so that it is not overfitting neither is it underfitting

#### Fourier transforms####

- eliminate noise and create approximations of the real stock movement.
- creates a series of sine waves with different amplitudes and frames. Wen combined, these sine waves approximate the original function. 
- we use them to extract global and local trends in the stock price.

#### Stacked autoencoders ####

- to find the missing correlations, we can find new types of features that affect stock movements. 
- Autoencode is a neural network model that seeks to learn a compressed representation of an input.
    - They are unsupervised but are trained with supervised learning methods. They are typically trained as part of a broader model that attemps to recreate the input.
    - They have a design with a bottleneck at the midpoint from which the reconstruction of the input data is performed. 
    - The most commong use isa learned or automatic feature extraction model.
    - We can take the data up until the bottleneck and use that as a fixed length vector that provides a compressed representation of hte input data, can be used for supervised learning model, visualization or more generally for dimensionality reduction.
  #### LSTM Autoencoder:
      - Implementation of autoencoder for sequence data usin and Encoder- Decoder LSTM architecture.
      - an encoder-decoder LSTM is configured to read the input sequence, encode it, decode it, and recreate it. The performance is evaluated based on the model's ability to recreate the input sequence.
      - When the model has a desired level of performance, the decoder part of the model may be removed and we will have just hte encoder part. 
      - Composite model: two decoders used to predict the next frame in the sequence and one to reconstruct frames in the sequence. 

##### steps:
- perform statistical checks for the 'quality' of the data. Make sure the data does not have heteroskedasticity, multicollinearity, or serial correlation.
- create feature importance, use XGBoost to do that.
- Heteroskedasticity: when the difference between predicted value by a regression and the real value is dependent on the data i.e. error grows when the data point grows along the x-axis.
- Serial Correlation: when one data is a formula of another feature.

##### PCA:
- reduce the dimensionality of the features created from autoencoders, using eigen portfolios



In [1]:
# To be crystal clear, the top of your code file must have the following 4 lines before any others;
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

In [2]:
!pip install python-decouple

In [3]:
import pandas as pd
import numpy as np
import quandl
import datetime
from decouple import config
import math
# sklearn imports
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import matplotlib.pyplot as plt
# keras imports
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers import RepeatVector, TimeDistributed
# notebook imports
from fin_data_fundamentals import find_fundamentals
from fin_data_fundamentals import get_fundamentals
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries
from fin_data import DailyTimeSeries
from fracdiff import FractionalDifferentiation as fd

Using TensorFlow backend.


In [4]:
from fin_data import DailyTimeSeries
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Let's pull in the data:

In [5]:
# We will experiment with Tesla stock

visa = DailyTimeSeries('V')

df = visa.initiate()

################################################################### 
 Ticker:  V 
 Last Refreshed:  2019-09-20 
 Data Retrieved:  Daily Time Series with Splits and Dividend Events 
 ###################################################################


In [6]:
df.head()

,V open,V high,V low,V close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2008-03-19,59.50,69.00,55.00,56.50,11.1997,708486000.0,0.0,1.0
2008-03-20,58.40,65.00,57.50,64.35,12.7558,198985200.0,0.0,1.0
2008-03-24,67.24,67.70,59.00,59.73,11.8400,149566400.0,0.0,1.0
2008-03-25,60.58,64.25,59.82,63.25,12.5377,87092000.0,0.0,1.0
2008-03-26,62.73,64.48,61.57,63.96,12.6785,43111600.0,0.0,1.0


plt.figure(figsize=(20,12))

plt.plot(df['TSLA close'], color = 'teal')
plt.plot(df['5. adjusted close'], color = 'red');

#### Adding Indicators ##

Add indicators, technical, securities, and macro to the dataframe

In [7]:
# add the fundamentals,

fund_list = ["operatingrevenue", "totalrevenue", "netincome", "totaloperatingexpenses", "totalgrossprofit", "totaloperatingincome", "totalpretaxincome", "weightedavebasicdilutedsharesos", "cashdividendspershare", "totalcostofrevenue"]

# not a complete list of fundamentals that I can pull in from the API, but this is good so far.

In [8]:
df_fund = visa.add_fundamentals(df, fund_list)

################################################################### 
 Ticker:  V 
 Fundamentals Retrieved:  ['V open' 'V high' 'V low' 'V close' '5. adjusted close' '6. volume'
 '7. dividend amount' '8. split coefficient' 'V_operatingrevenue'
 'V_totalrevenue' 'V_netincome' 'V_totaloperatingexpenses'
 'V_totalgrossprofit' 'V_totaloperatingincome' 'V_totalpretaxincome'
 'V_weightedavebasicdilutedsharesos' 'V_cashdividendspershare'
 'V_totalcostofrevenue'] 
 ###################################################################
################################################################### 
 Ticker:  V 
 Retrieved Data Start Date:  2009-07-30 
 Retrieved Data End Date:  2019-07-26 
 Data Retrieved:  ['V_operatingrevenue', 'V_totalrevenue', 'V_netincome', 'V_totaloperatingexpenses', 'V_totalgrossprofit', 'V_totaloperatingincome', 'V_totalpretaxincome', 'V_weightedavebasicdilutedsharesos', 'V_cashdividendspershare', 'V_totalcostofrevenue'] 
 ##############################################

In [9]:
df_fund.head()

,V open,V high,V low,V close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,V_operatingrevenue,V_totalrevenue,V_netincome,V_totaloperatingexpenses,V_totalgrossprofit,V_totaloperatingincome,V_totalpretaxincome,V_weightedavebasicdilutedsharesos,V_cashdividendspershare,V_totalcostofrevenue
date,,,,,,,,,,,,,,,,,,
2008-03-19,59.50,69.00,55.00,56.50,11.1997,708486000.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-03-20,58.40,65.00,57.50,64.35,12.7558,198985200.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-03-24,67.24,67.70,59.00,59.73,11.8400,149566400.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-03-25,60.58,64.25,59.82,63.25,12.5377,87092000.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-03-26,62.73,64.48,61.57,63.96,12.6785,43111600.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0


In [10]:
# Add data with all the indicators:

def tech_company_data(ticker):
    """
    Containes Tech Index
    """    
    dts = DailyTimeSeries(ticker)
    df = dts.initiate()
    with HiddenPrints():
        df = dts.add_securities(['XLK', 'vix', 'SPX'], primary_df=df)
        df = dts.add_technicals(['SMA', 'EMA', 'MACD', 'STOCH', 
                                   'RSI', 'ADX', 'CCI', 'BBANDS', 
                                   'AD', 'OBV'], 
                                  primary_df=df)
        df = dts.add_macro(primary_df=df, 
                             indices=['housing_index', 'confidence_index', 'trade_index', 'longterm_rates'])
    
    return df

In [11]:
df = tech_company_data('V')


################################################################### 
 Ticker:  V 
 Last Refreshed:  2019-09-20 
 Data Retrieved:  Daily Time Series with Splits and Dividend Events 
 ###################################################################


C:\Users\Billi\repos\market_makers\Data-Science\data\fin_data.py:296: UserWarning: The latest value available for Housing Index is from January 2019.
  warnings.warn("The latest value available for Housing Index is from January 2019.")


In [12]:
df.head()

,V open,V high,V low,V close,5. adjusted close_x,6. volume_x,7. dividend amount_x,8. split coefficient_x,XLK open,XLK high,XLK low,XLK close,5. adjusted close_y,6. volume_y,7. dividend amount_y,8. split coefficient_y,vix open,vix high,vix low,vix close,5. adjusted close_x,6. volume_x,7. dividend amount_x,8. split coefficient_x,SPX open,SPX high,SPX low,SPX close,5. adjusted close_y,6. volume_y,7. dividend amount_y,8. split coefficient_y,V_SMA,V_EMA,V_MACD,V_MACD_Signal,V_MACD_Hist,V_SlowD,V_SlowK,V_RSI,V_ADX,V_CCI,V_Real Middle Band,V_Real Upper Band,V_Real Lower Band,V_Chaikin A/D,V_OBV,housing_index,conf_index,conf_index_SE,trade_value,10 Yrs Rates,20-Yr Maturity Rate
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-05-14,83.2949,84.35,81.45,82.23,16.3001,51400400.0,0.0,1.0,25.020,25.31,24.94,24.95,20.7378,4226000.0,0.0,1.0,17.98,17.98,16.74,17.66,17.66,0.0,0.0,1.0,1405.65,1420.1899,1405.65,1408.6600,1408.6600,3.979370e+09,0.0,1.0,79.0320,79.3058,4.8159,5.2011,-0.3851,35.8379,34.7584,65.1019,43.8072,40.6145,79.0320,92.6461,65.4179,-3.194883e+08,1.489803e+09,167.328,63.53,2.61,96.2716,4.49,4.63
2008-05-15,82.5300,82.93,81.11,82.25,16.3040,36688800.0,0.0,1.0,24.970,25.48,24.94,25.43,21.1368,2793000.0,0.0,1.0,17.65,17.84,16.25,16.30,16.30,0.0,0.0,1.0,1408.36,1424.4000,1406.87,1423.5699,1423.5699,3.836480e+09,0.0,1.0,79.7925,79.5862,4.4788,5.0566,-0.5779,35.1165,40.1106,65.1223,42.0522,28.9602,79.7925,92.2961,67.2889,-3.102153e+08,1.526492e+09,167.328,63.53,2.61,96.2716,4.41,4.55
2008-05-16,82.7100,82.85,81.26,82.37,16.3278,34066800.0,0.0,1.0,25.500,25.50,25.15,25.37,21.0869,3631700.0,0.0,1.0,16.30,17.92,16.30,16.47,16.47,0.0,0.0,1.0,1423.89,1425.8199,1414.35,1425.3500,1425.3500,3.842590e+09,0.0,1.0,80.4610,79.8513,4.1731,4.8799,-0.7068,41.7123,50.2678,65.2512,40.3849,24.4910,80.4610,91.9755,68.9465,-2.967171e+08,1.560558e+09,167.328,63.53,2.61,96.2716,4.44,4.57
2008-05-19,82.2700,84.74,82.15,83.32,16.5161,45473200.0,0.0,1.0,25.482,25.69,25.26,25.50,21.1950,4212200.0,0.0,1.0,16.47,17.89,15.82,17.01,17.01,0.0,0.0,1.0,1425.28,1440.2400,1421.63,1426.6300,1426.6300,3.683970e+09,0.0,1.0,81.0785,80.1817,3.9619,4.6963,-0.7344,50.0598,59.8011,66.2894,39.0943,36.9569,81.0785,91.7873,70.3697,-3.011064e+08,1.606032e+09,167.328,63.53,2.61,96.2716,4.42,4.55
2008-05-20,82.8800,83.84,81.91,82.74,16.4012,40702000.0,0.0,1.0,25.250,25.25,24.84,25.10,20.8625,3115200.0,0.0,1.0,17.02,18.42,17.02,17.58,17.58,0.0,0.0,1.0,1424.49,1424.4900,1409.09,1413.4000,1413.4000,3.854320e+09,0.0,1.0,81.7015,80.4253,3.7050,4.4981,-0.7931,56.2321,58.6274,65.0406,37.8194,22.8959,81.7015,91.2071,72.1959,-3.068005e+08,1.565330e+09,167.328,63.53,2.61,96.2716,4.38,4.52


In [13]:

df.shape

(2698, 53)

In [14]:
df = tech_company_data('V')

################################################################### 
 Ticker:  V 
 Last Refreshed:  2019-09-20 
 Data Retrieved:  Daily Time Series with Splits and Dividend Events 
 ###################################################################


C:\Users\Billi\repos\market_makers\Data-Science\data\fin_data.py:296: UserWarning: The latest value available for Housing Index is from January 2019.
  warnings.warn("The latest value available for Housing Index is from January 2019.")


In [15]:
df.tail()

,V open,V high,V low,V close,5. adjusted close_x,6. volume_x,7. dividend amount_x,8. split coefficient_x,XLK open,XLK high,XLK low,XLK close,5. adjusted close_y,6. volume_y,7. dividend amount_y,8. split coefficient_y,vix open,vix high,vix low,vix close,5. adjusted close_x,6. volume_x,7. dividend amount_x,8. split coefficient_x,SPX open,SPX high,SPX low,SPX close,5. adjusted close_y,6. volume_y,7. dividend amount_y,8. split coefficient_y,V_SMA,V_EMA,V_MACD,V_MACD_Hist,V_MACD_Signal,V_SlowK,V_SlowD,V_RSI,V_ADX,V_CCI,V_Real Middle Band,V_Real Upper Band,V_Real Lower Band,V_Chaikin A/D,V_OBV,housing_index,conf_index,conf_index_SE,trade_value,10 Yrs Rates,20-Yr Maturity Rate
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-25,139.00,139.900,137.97,138.67,138.0167,9756300.0,0.0,1.0,65.5000,66.125,65.2600,65.93,65.4863,10030800.0,0.0,1.0,18.43,18.46,17.31,17.42,17.42,0.0,0.0,1.0,2657.4399,2672.3799,2657.3301,2664.7600,2664.7600,3.814080e+09,0.0,1.0,135.8290,136.4451,0.9699,0.3524,0.6175,46.2690,50.7652,53.7618,12.1218,79.0098,135.8290,141.8311,129.8269,5.932097e+08,2.768893e+09,205.073,54.48,4.30,126.8701,2.99,2.92
2019-01-28,137.51,137.510,134.81,135.99,135.3493,9857200.0,0.0,1.0,64.9500,65.070,64.5100,65.05,64.6122,9662400.0,0.0,1.0,18.56,20.42,18.42,18.87,18.87,0.0,0.0,1.0,2644.9700,2644.9700,2624.0601,2643.8501,2643.8501,3.612810e+09,0.0,1.0,136.0280,136.4017,0.8029,0.1483,0.6546,45.4754,45.8219,49.6072,11.8062,4.2198,136.0280,141.7686,130.2874,5.919684e+08,2.759036e+09,205.073,54.48,4.30,126.8701,2.99,2.92
2019-01-29,136.59,136.690,134.11,135.00,134.3640,7457700.0,0.0,1.0,65.1863,65.190,64.2500,64.35,63.9169,8573900.0,0.0,1.0,19.45,19.93,18.42,19.13,19.13,0.0,0.0,1.0,2644.8899,2650.9299,2631.0500,2640.0000,2640.0000,3.504200e+09,0.0,1.0,136.2310,136.2682,0.5840,-0.0564,0.6405,35.2572,42.3339,48.1601,11.6143,-28.8534,136.2310,141.5058,130.9562,5.896560e+08,2.751578e+09,205.073,54.48,4.30,126.8701,2.97,2.90
2019-01-30,136.10,137.895,135.51,137.60,136.9517,8078100.0,0.0,1.0,65.4200,66.540,65.1298,66.35,65.9035,16995300.0,0.0,1.0,19.15,19.31,17.54,17.66,17.66,0.0,0.0,1.0,2653.6201,2690.4399,2648.3401,2681.0500,2681.0500,3.857810e+09,0.0,1.0,136.5140,136.3951,0.6133,-0.0218,0.6350,32.9435,37.8920,52.0288,11.2168,22.0824,136.5140,141.4329,131.5951,5.957357e+08,2.759656e+09,205.073,54.48,4.30,126.5041,2.98,2.90
2019-01-31,134.39,135.730,133.30,135.01,134.3739,20095700.0,0.0,1.0,66.0100,66.640,65.8000,66.28,65.8339,18250900.0,0.0,1.0,17.39,17.72,16.54,16.57,16.57,0.0,0.0,1.0,2685.4900,2708.9500,2678.6499,2704.1001,2704.1001,4.917650e+09,0.0,1.0,136.6185,136.2631,0.4226,-0.1700,0.5926,33.8523,34.0176,48.2528,11.1833,-79.8969,136.6185,141.3111,131.9259,6.039229e+08,2.739561e+09,204.708,47.93,3.84,126.5041,2.91,2.83


In [16]:
# merge the df and df_fund data together

df_new = pd.merge(df, df_fund, on='date')

df_new.head()


,V open_x,V high_x,V low_x,V close_x,5. adjusted close_x,6. volume_x,7. dividend amount_x,8. split coefficient_x,XLK open,XLK high,XLK low,XLK close,5. adjusted close_y,6. volume_y,7. dividend amount_y,8. split coefficient_y,vix open,vix high,vix low,vix close,5. adjusted close_x,6. volume_x,7. dividend amount_x,8. split coefficient_x,SPX open,SPX high,SPX low,SPX close,5. adjusted close_y,6. volume_y,7. dividend amount_y,8. split coefficient_y,V_SMA,V_EMA,V_MACD,V_MACD_Hist,V_MACD_Signal,V_SlowK,V_SlowD,V_RSI,V_ADX,V_CCI,V_Real Middle Band,V_Real Upper Band,V_Real Lower Band,V_Chaikin A/D,V_OBV,housing_index,conf_index,conf_index_SE,trade_value,10 Yrs Rates,20-Yr Maturity Rate,V open_y,V high_y,V low_y,V close_y,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,V_operatingrevenue,V_totalrevenue,V_netincome,V_totaloperatingexpenses,V_totalgrossprofit,V_totaloperatingincome,V_totalpretaxincome,V_weightedavebasicdilutedsharesos,V_cashdividendspershare,V_totalcostofrevenue
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-05-14,83.2949,84.35,81.45,82.23,16.3001,51400400.0,0.0,1.0,25.020,25.31,24.94,24.95,20.7378,4226000.0,0.0,1.0,17.98,17.98,16.74,17.66,17.66,0.0,0.0,1.0,1405.65,1420.1899,1405.65,1408.6600,1408.6600,3.979370e+09,0.0,1.0,79.0320,79.3058,4.8159,-0.3851,5.2011,34.7584,35.8379,65.1019,43.8072,40.6145,79.0320,92.6461,65.4179,-3.194883e+08,1.489803e+09,167.328,63.53,2.61,96.2716,4.49,4.63,83.2949,84.35,81.45,82.23,16.3001,51400400.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-05-15,82.5300,82.93,81.11,82.25,16.3040,36688800.0,0.0,1.0,24.970,25.48,24.94,25.43,21.1368,2793000.0,0.0,1.0,17.65,17.84,16.25,16.30,16.30,0.0,0.0,1.0,1408.36,1424.4000,1406.87,1423.5699,1423.5699,3.836480e+09,0.0,1.0,79.7925,79.5862,4.4788,-0.5779,5.0566,40.1106,35.1165,65.1223,42.0522,28.9602,79.7925,92.2961,67.2889,-3.102153e+08,1.526492e+09,167.328,63.53,2.61,96.2716,4.41,4.55,82.5300,82.93,81.11,82.25,16.3040,36688800.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-05-16,82.7100,82.85,81.26,82.37,16.3278,34066800.0,0.0,1.0,25.500,25.50,25.15,25.37,21.0869,3631700.0,0.0,1.0,16.30,17.92,16.30,16.47,16.47,0.0,0.0,1.0,1423.89,1425.8199,1414.35,1425.3500,1425.3500,3.842590e+09,0.0,1.0,80.4610,79.8513,4.1731,-0.7068,4.8799,50.2678,41.7123,65.2512,40.3849,24.4910,80.4610,91.9755,68.9465,-2.967171e+08,1.560558e+09,167.328,63.53,2.61,96.2716,4.44,4.57,82.7100,82.85,81.26,82.37,16.3278,34066800.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-05-19,82.2700,84.74,82.15,83.32,16.5161,45473200.0,0.0,1.0,25.482,25.69,25.26,25.50,21.1950,4212200.0,0.0,1.0,16.47,17.89,15.82,17.01,17.01,0.0,0.0,1.0,1425.28,1440.2400,1421.63,1426.6300,1426.6300,3.683970e+09,0.0,1.0,81.0785,80.1817,3.9619,-0.7344,4.6963,59.8011,50.0598,66.2894,39.0943,36.9569,81.0785,91.7873,70.3697,-3.011064e+08,1.606032e+09,167.328,63.53,2.61,96.2716,4.42,4.55,82.2700,84.74,82.15,83.32,16.5161,45473200.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-05-20,82.8800,83.84,81.91,82.74,16.4012,40702000.0,0.0,1.0,25.250,25.25,24.84,25.10,20.8625,3115200.0,0.0,1.0,17.02,18.42,17.02,17.58,17.58,0.0,0.0,1.0,1424.49,1424.4900,1409.09,1413.4000,1413.4000,3.854320e+09,0.0,1.0,81.7015,80.4253,3.7050,-0.7931,4.4981,58.6274,56.2321,65.0406,37.8194,22.8959,81.7015,91.2071,72.1959,-3.068005e+08,1.565330e+09,167.328,63.53,2.61,96.2716,4.38,4.52,82.8800,83.84,81.91,82.74,16.4012,40702000.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0


Finally, the new dataframe has all the columns and indicators that we were initially planning on using. Let's move forward from here.

In [17]:
print('There are {} number of days in the dataset.'.format(df_new.shape[0]))

There are 2698 number of days in the dataset.


### Target & Split

We will use the adjusted close shift by 1 day for now.

We will then split the dataframe into X and y and further training and testing sets.

In [19]:
#df['percent_change'] = ((df['5. adjusted close_x'].shift(-1) - df['TSLA close']) / (df['TSLA close']))*100


In [20]:
#df['percent_change'].head()

date
2010-08-24    3.645833
2010-08-25   -0.753769
2010-08-26   -0.253165
2010-08-27    0.862944
2010-08-30   -1.962758
Name: percent_change, dtype: float64

In [20]:
df_new['new'] = (df_new['5. adjusted close_x'].shift(-1))
                     # - df_new['TSLA close_x']) / (df_new['TSLA close_x']))*100


ValueError: Wrong number of items passed 2, placement implies 1

In [74]:
df_new['target'].head()

date
2010-08-24    19.90
2010-08-25    19.75
2010-08-26    19.70
2010-08-27    19.87
2010-08-30    19.48
Name: target, dtype: float64

In [21]:
df_new.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['V open_x', 'V high_x', 'V low_x', 'V close_x', '5. adjusted close_x',
       '6. volume_x', '7. dividend amount_x', '8. split coefficient_x',
       'XLK open', 'XLK high', 'XLK low', 'XLK close', '5. adjusted close_y',
       '6. volume_y', '7. dividend amount_y', '8. split coefficient_y',
       'vix open', 'vix high', 'vix low', 'vix close', '5. adjusted close_x',
       '6. volume_x', '7. dividend amount_x', '8. split coefficient_x',
       'SPX open', 'SPX high', 'SPX low', 'SPX close', '5. adjusted close_y',
       '6. volume_y', '7. dividend amount_y', '8. split coefficient_y',
       'V_SMA', 'V_EMA', 'V_MACD', 'V_MACD_Hist', 'V_MACD_Signal', 'V_SlowK',
       'V_SlowD', 'V_RSI', 'V_ADX', 'V_CCI', 'V_Real Middle Band',
       'V_Real Upper Band', 'V_Real Lower Band', 'V_Chaikin A/D', 'V_OBV',
       'housing_index', 'conf_index', 'conf_index_SE', 'trade_value',
       '10 Yrs Rates', '20-Yr Maturity Rate', 'V open_y', 'V high_y',
  

In [76]:
df_new.head()

,TSLA open_x,TSLA high_x,TSLA low_x,TSLA close_x,5. adjusted close_x,6. volume_x,7. dividend amount_x,8. split coefficient_x,XLK open,XLK high,XLK low,XLK close,5. adjusted close_y,6. volume_y,7. dividend amount_y,8. split coefficient_y,vix open,vix high,vix low,vix close,5. adjusted close_x,6. volume_x,7. dividend amount_x,8. split coefficient_x,SPX open,SPX high,SPX low,SPX close,5. adjusted close_y,6. volume_y,7. dividend amount_y,8. split coefficient_y,TSLA_SMA,TSLA_EMA,TSLA_MACD_Hist,TSLA_MACD,TSLA_MACD_Signal,TSLA_SlowK,TSLA_SlowD,TSLA_RSI,TSLA_ADX,TSLA_CCI,TSLA_Real Middle Band,TSLA_Real Upper Band,TSLA_Real Lower Band,TSLA_Chaikin A/D,TSLA_OBV,housing_index,conf_index,conf_index_SE,trade_value,10 Yrs Rates,20-Yr Maturity Rate,TSLA open_y,TSLA high_y,TSLA low_y,TSLA close_y,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,TSLA_operatingrevenue,TSLA_totalrevenue,TSLA_netincome,TSLA_totaloperatingexpenses,TSLA_totalgrossprofit,TSLA_totaloperatingincome,TSLA_totalpretaxincome,TSLA_weightedavebasicdilutedsharesos,TSLA_cashdividendspershare,TSLA_totalcostofrevenue,target
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-08-24,19.25,19.710,18.95,19.20,19.20,673100.0,0.0,1.0,20.93,21.079,20.77,20.87,17.9672,10590300.0,0.0,1.0,27.91,28.77,26.32,27.46,27.46,0.0,0.0,1.0,1063.20,1063.20,1046.6801,1051.87,1051.87,4.436330e+09,0.0,1.0,19.5775,19.3641,-0.0444,-0.1483,-0.1039,64.2590,65.1365,43.3780,4.5065,-21.9384,19.5775,21.7879,17.3671,-5.397985e+06,-12174000.0,147.552,63.87,3.48,102.4331,3.17,3.28,19.25,19.710,18.95,19.20,19.20,673100.0,0.0,1.0,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-34852000.0,92270721.0,0.0,21945000.0,19.90
2010-08-25,19.16,19.980,18.56,19.90,19.90,503300.0,0.0,1.0,20.74,21.090,20.69,21.00,18.0791,11052900.0,0.0,1.0,28.30,28.92,26.46,26.70,26.70,0.0,0.0,1.0,1048.98,1059.38,1039.8300,1055.33,1055.33,4.360190e+09,0.0,1.0,19.5365,19.4152,0.0149,-0.0853,-0.1001,68.6417,67.5305,46.1348,4.5061,-3.2187,19.5365,21.6903,17.3827,-4.951395e+06,-11670700.0,147.552,63.87,3.48,102.9749,3.20,3.31,19.16,19.980,18.56,19.90,19.90,503300.0,0.0,1.0,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-34852000.0,92270721.0,0.0,21945000.0,19.75
2010-08-26,19.89,20.270,19.60,19.75,19.75,433800.0,0.0,1.0,21.10,21.130,20.76,20.79,17.8983,10154700.0,0.0,1.0,26.45,27.55,25.86,27.37,27.37,0.0,0.0,1.0,1056.28,1061.45,1045.4000,1047.22,1047.22,3.646710e+09,0.0,1.0,19.5065,19.4471,0.0426,-0.0469,-0.0895,61.4989,64.7999,45.6336,4.6097,31.2156,19.5065,21.6307,17.3823,-5.190957e+06,-12104500.0,147.552,63.87,3.48,102.9749,3.15,3.25,19.89,20.270,19.60,19.75,19.75,433800.0,0.0,1.0,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-34852000.0,92270721.0,0.0,21945000.0,19.70
2010-08-27,19.75,19.865,19.50,19.70,19.70,379600.0,0.0,1.0,20.91,21.050,20.58,21.04,18.1136,14991700.0,0.0,1.0,26.50,28.11,24.41,24.45,24.45,0.0,0.0,1.0,1049.27,1065.21,1039.7000,1064.59,1064.59,4.102460e+09,0.0,1.0,19.4945,19.4711,0.0554,-0.0203,-0.0757,68.1908,66.1105,45.4603,4.6664,17.5294,19.4945,21.6114,17.3776,-5.154557e+06,-12484100.0,147.552,63.87,3.48,102.9749,3.31,3.41,19.75,19.865,19.50,19.70,19.70,379600.0,0.0,1.0,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-34852000.0,92270721.0,0.0,21945000.0,19.87
2010-08-30,19.70,20.190,19.61,19.87,19.87,732800.0,0.0,1.0,20.93,21.090,20.76,20.76,17.8725,7593500.0,0.0,1.0,25.88,27.21,25.41,27.21,27.21,0.0,0.0,1.0,1062.90,1064.40,1048.7900,1048.92,1048.92,2.917990e+09,0.0,1.0,19.4420,19.5091,0.0720,0.0144,-0.0576,68.3100,65.9999,46.1916,4.8434,40.1607,19.4420,21.4649,17.4191,-5.230363e+06,-11751300.0,147.552,63.87,3.48,102.9749,3.19,3.30,19.70,20.190,19.61,19.87,19.87,732800.0,0.0,1.0,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-34852000.0,92270721.0,0.0,21945000.0,19.48


df.columns

In [24]:
df_new.columns = ['VISA open', 'VISA high', 'VISA low', 'VISA close',
       'VISA adj_close', 'VISA_vol', 'VISA_div',
       'VISA_coef', 'XLK open', 'XLK high', 'XLK low',
       'XLK close', 'XLK adj_close', 'XLK_vol',
       'XLK_div', 'XLK_coef', 'vix open',
       'vix high', 'vix low', 'vix close', 'vix adj_close',
       'vix_vol', 'vix_div', 'vix_coef',
       'SPX open', 'SPX high', 'SPX low', 'SPX close', 'SPX adj_close',
       'SPX vol', 'SPX_div', 'SPX_coef',
       'VISA_SMA', 'VISA_EMA', 'VISA_MACD_Signal', 'VISA_MACD_Hist',
       'VISA_MACD', 'VISA_SlowD', 'VISA_SlowK', 'VISA_RSI', 'VISA_ADX',
       'VISA_CCI', 'VISA_Real Middle Band', 'VISA_Real Upper Band',
       'VISA_Real Lower Band', 'VISA_Chaikin A/D', 'VISA_OBV', 'housing_index',
       'conf_index', 'conf_index_SE', 'trade_value', '10 Yrs Rates',
       '20-Yr Maturity Rate', 'VISA open_y', 'VISA high_y', 'VISA low_y',
       'VISA close_y', '5. adjusted close', '6. volume', '7. dividend amount',
       '8. split coefficient', 'VISA_operatingrevenue', 'VISA_totalrevenue',
       'VISA_netincome', 'VISA_totaloperatingexpenses',
       'VISA_totalgrossprofit', 'VISA_totaloperatingincome', "totalpretaxincome", "weightedavebasicdilutedsharesos", "cashdividendspershare", "totalcostofrevenue"]

In [25]:
df_new.head()

,VISA open,VISA high,VISA low,VISA close,VISA adj_close,VISA_vol,VISA_div,VISA_coef,XLK open,XLK high,XLK low,XLK close,XLK adj_close,XLK_vol,XLK_div,XLK_coef,vix open,vix high,vix low,vix close,vix adj_close,vix_vol,vix_div,vix_coef,SPX open,SPX high,SPX low,SPX close,SPX adj_close,SPX vol,SPX_div,SPX_coef,VISA_SMA,VISA_EMA,VISA_MACD_Signal,VISA_MACD_Hist,VISA_MACD,VISA_SlowD,VISA_SlowK,VISA_RSI,VISA_ADX,VISA_CCI,VISA_Real Middle Band,VISA_Real Upper Band,VISA_Real Lower Band,VISA_Chaikin A/D,VISA_OBV,housing_index,conf_index,conf_index_SE,trade_value,10 Yrs Rates,20-Yr Maturity Rate,VISA open_y,VISA high_y,VISA low_y,VISA close_y,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,VISA_operatingrevenue,VISA_totalrevenue,VISA_netincome,VISA_totaloperatingexpenses,VISA_totalgrossprofit,VISA_totaloperatingincome,totalpretaxincome,weightedavebasicdilutedsharesos,cashdividendspershare,totalcostofrevenue
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-05-14,83.2949,84.35,81.45,82.23,16.3001,51400400.0,0.0,1.0,25.020,25.31,24.94,24.95,20.7378,4226000.0,0.0,1.0,17.98,17.98,16.74,17.66,17.66,0.0,0.0,1.0,1405.65,1420.1899,1405.65,1408.6600,1408.6600,3.979370e+09,0.0,1.0,79.0320,79.3058,4.8159,-0.3851,5.2011,34.7584,35.8379,65.1019,43.8072,40.6145,79.0320,92.6461,65.4179,-3.194883e+08,1.489803e+09,167.328,63.53,2.61,96.2716,4.49,4.63,83.2949,84.35,81.45,82.23,16.3001,51400400.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-05-15,82.5300,82.93,81.11,82.25,16.3040,36688800.0,0.0,1.0,24.970,25.48,24.94,25.43,21.1368,2793000.0,0.0,1.0,17.65,17.84,16.25,16.30,16.30,0.0,0.0,1.0,1408.36,1424.4000,1406.87,1423.5699,1423.5699,3.836480e+09,0.0,1.0,79.7925,79.5862,4.4788,-0.5779,5.0566,40.1106,35.1165,65.1223,42.0522,28.9602,79.7925,92.2961,67.2889,-3.102153e+08,1.526492e+09,167.328,63.53,2.61,96.2716,4.41,4.55,82.5300,82.93,81.11,82.25,16.3040,36688800.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-05-16,82.7100,82.85,81.26,82.37,16.3278,34066800.0,0.0,1.0,25.500,25.50,25.15,25.37,21.0869,3631700.0,0.0,1.0,16.30,17.92,16.30,16.47,16.47,0.0,0.0,1.0,1423.89,1425.8199,1414.35,1425.3500,1425.3500,3.842590e+09,0.0,1.0,80.4610,79.8513,4.1731,-0.7068,4.8799,50.2678,41.7123,65.2512,40.3849,24.4910,80.4610,91.9755,68.9465,-2.967171e+08,1.560558e+09,167.328,63.53,2.61,96.2716,4.44,4.57,82.7100,82.85,81.26,82.37,16.3278,34066800.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-05-19,82.2700,84.74,82.15,83.32,16.5161,45473200.0,0.0,1.0,25.482,25.69,25.26,25.50,21.1950,4212200.0,0.0,1.0,16.47,17.89,15.82,17.01,17.01,0.0,0.0,1.0,1425.28,1440.2400,1421.63,1426.6300,1426.6300,3.683970e+09,0.0,1.0,81.0785,80.1817,3.9619,-0.7344,4.6963,59.8011,50.0598,66.2894,39.0943,36.9569,81.0785,91.7873,70.3697,-3.011064e+08,1.606032e+09,167.328,63.53,2.61,96.2716,4.42,4.55,82.2700,84.74,82.15,83.32,16.5161,45473200.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0
2008-05-20,82.8800,83.84,81.91,82.74,16.4012,40702000.0,0.0,1.0,25.250,25.25,24.84,25.10,20.8625,3115200.0,0.0,1.0,17.02,18.42,17.02,17.58,17.58,0.0,0.0,1.0,1424.49,1424.4900,1409.09,1413.4000,1413.4000,3.854320e+09,0.0,1.0,81.7015,80.4253,3.7050,-0.7931,4.4981,58.6274,56.2321,65.0406,37.8194,22.8959,81.7015,91.2071,72.1959,-3.068005e+08,1.565330e+09,167.328,63.53,2.61,96.2716,4.38,4.52,82.8800,83.84,81.91,82.74,16.4012,40702000.0,0.0,1.0,0.0,1.613000e+09,422000000.0,0.0,0.0,0.0,714000000.0,3.516700e+09,0.0,0.0


In [26]:
df_new['target'] = df_new['VISA adj_close'].shift(-1)

In [28]:
X = df_new.drop(columns=['target', 'VISA close', 'VISA adj_close', 'VISA open', 'VISA high', 'VISA low'])
y = df_new[['target']].values

In [29]:
X.shape, y.shape

((2698, 66), (2698, 1))

In [30]:
def prep_data(train_cut, val_cut, X=X, y=y):
    
    scaler = MinMaxScaler()
    scaler.fit(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        train_size=train_cut, 
                                                        shuffle=False)

    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test,
                                                    train_size=val_cut,
                                                    shuffle=False)
    
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_val = scaler.transform(X_val)
    
    return X_train, X_test, X_val, y_train, y_test, y_val

In [31]:
X_train, X_test, X_val, y_train, y_test, y_val = prep_data(train_cut=0.8, val_cut= 0.6)

In [32]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_val.shape, y_val.shape

((2158, 66), (324, 66), (2158, 1), (324, 1), (216, 66), (216, 1))

##### Time Series Generator

In [33]:
train_data_generator = TimeseriesGenerator(X_train, y_train, 
                                           length=15,
                                           sampling_rate=1, 
                                           stride=1, 
                                           batch_size=6)

test_data_generator = TimeseriesGenerator(X_test, y_test, 
                                          length=15, 
                                          sampling_rate=1,
                                          stride=1,
                                          batch_size=6)

val_data_generator = TimeseriesGenerator(X_val, y_val, 
                                          length=15, 
                                          sampling_rate=1,
                                          stride=1,
                                          batch_size=6)

### Model 

In [34]:
# imports

from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import h5py


In [35]:
model = Sequential()
model.add(LSTM(256, activation='relu', 
               return_sequences=True, 
               input_shape=(train_data_generator.length, 
                            X_train.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(144, return_sequences=True,
               activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(72, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

W0922 17:48:11.445656   652 deprecation_wrapper.py:119] From C:\Users\Billi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0922 17:48:11.447651   652 deprecation_wrapper.py:119] From C:\Users\Billi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0922 17:48:11.455628   652 deprecation_wrapper.py:119] From C:\Users\Billi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0922 17:48:11.806285   652 deprecation_wrapper.py:119] From C:\Users\Billi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0922 17:48:11.814264   652 deprecation.py:506

In [36]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('update-weights-{epoch:02d}-{val_loss:.2f}.hdf5',
                      monitor='val_loss',
                      mode='min',
                      verbose=1,
                      save_best_only=True)


In [32]:
history = model.fit_generator(train_data_generator,
                              epochs =200,
                              validation_data = test_data_generator,
                              verbose=1,
                              callbacks = [es, mc])

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
281/281 [==============================] - 25s 89ms/step - loss: 37103.4787 - val_loss: 5111.3470
Epoch 2/100
281/281 [==============================] - 21s 75ms/step - loss: 1973.2354 - val_loss: 985.1973
Epoch 3/100
281/281 [==============================] - 23s 83ms/step - loss: 1091.9786 - val_loss: 1239.6058
Epoch 4/100
281/281 [==============================] - 23s 83ms/step - loss: 1252.4138 - val_loss: 988.3753
Epoch 5/100
281/281 [==============================] - 21s 75ms/step - loss: 1041.1830 - val_loss: 736.3423
Epoch 6/100
281/281 [==============================] - 24s 86ms/step - loss: 841.1205 - val_loss: 1143.2841
Epoch 7/100
281/281 [==============================] - 25s 89ms/step - loss: 929.4206 - val_loss: 1065.0535
Epoch 8/100
281/281 [==============================] - 26s 92ms/step - loss: 838.9474 - val_loss: 717.3310
Epoch 9/100
281/281 [==============================] - 21s 76ms/step - loss: 866.8625 

In [ ]:
# pickle the model:
with open('model_tesla.pickle', 'wb') as mod_f:
    pickle.dump(model, mod_f)
    
    

In [ ]:
with open('model_tesla.pickle', 'rb') as mod_f:
    model = pickle.load(mod_fr)
    

In [ ]:
model.load_weights('weights-improvement.hdf5')

In [47]:
# plot the training
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()


In [49]:
plt.figure(figsize=(20,12))
y_pred = model.predict_generator(test_data_generator)
for i in range(30):
    y_pred = np.insert(y_pred, 0, [0], axis=0)
    
plt.plot(y_pred)
plt.plot(y_test);


ValueError: could not broadcast input array from shape (240,15,1) into shape (240)

<Figure size 1440x864 with 0 Axes>

## Shapley

In [ ]:
from shap import DeepExplainer, summary_plot

In [92]:
def extract_data(generator):
    for i in np.arange(len(generator)):
        if i == 0:
            a, b = generator[i]
        else: 
            c, d = generator[i]

            a = np.vstack((a, c))
            b = np.vstack((b, d))
            
    return a, b

In [93]:
def individual_array(sample):
    
    # get the means for each sampling
    for i in np.arange(len(sample[0])):
        means = []
        for j in np.arange(len (sample)):
            means.append(sample[j][i])
        s_value = np.array(np.mean(means))

        if i == 0:
            final_array = s_value
        else:
            final_array = np.hstack((final_array, s_value))
            
    return final_array

In [94]:
def flatten_shap_values(shap_vals):
    
    #Pull the array out of the list. 
    sv = shap_vals[0]
    
    count = 0
    for sample in sv: 
        sample_array = individual_array(sample)
        if count == 0:
            final_array = sample_array
        else:
            final_array = np.vstack((final_array, sample_array))
        
        count +=1
        
    return final_array

In [95]:
a, b = extract_data(test_data_generator)

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
from shap import DeepExplainer, summary_plot

In [ ]:
de = DeepExplainer(model = model, data=a)

shaps_vals = de.shap_values(a)

In [ ]:
print(shaps_vals[0].shape)
print(shaps_vals[0][0].shape)
print(shaps_vals[0][0][0].shape)


In [ ]:
summary_plot([shaps_vals[0][100]], feature_names=df.columns)

In [ ]:
individual_array(shaps_vals[0][0])

In [ ]:
summary_lot([flatten_shap_values(shaps_vals)],
            feature_names = (df.drop(columns=['target'])
                             .columns
                             .tolist()))


In [ ]:
shap_df = pd.DataFrame(data=flatten_shap_values(shaps_vals),columns = df.drop(columns='target').columns.tolist())

In [ ]:
shap_df.head()

In [ ]:

np.save('shaps_vals', shaps_vals)
np.save('feature_gen_extracted', a)
np.save('target_gen_extracted', b)

In [ ]:
import pca_concordance as pcac

In [ ]:
pcac.feature_pca_concordance(a, shap_df)


In [ ]:
import dill

In [ ]:
shaps_vals[0]